### get dependencies

In [154]:
## Get dependencies ##

import numpy as np
import string
import math
import sys
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib
sys.path.append('..')
from GIR import *
import scipy as sp
import pickle
import time
import scipy as sp
from scipy import signal
from scipy.io.idl import readsav
import os
from mpl_toolkits.axes_grid1.inset_locator import inset_axes
import glob
import requests
import ftplib
import io
import cmocean
from bs4 import BeautifulSoup
import urllib.request
from io import StringIO, BytesIO
from zipfile import ZipFile
import tqdm
import seaborn as sn
import re

## get SSP emissions
from tools.RCMIP import *

header = {
  "User-Agent": "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/50.0.2661.75 Safari/537.36",
  "X-Requested-With": "XMLHttpRequest"
}
    
## Matplotlib rcparams setup:

matplotlib.rcParams['font.family']='Helvetica'
matplotlib.rcParams['font.size']=11
# matplotlib.rcParams['font.weight']=400

matplotlib.rcParams['image.cmap']='cmo.ice'

matplotlib.rcParams['axes.prop_cycle']=matplotlib.cycler('color',['011936','FF7D00','225560','BFACAA','D72638','788C9B','A33F00','7CAAB0','685655','EB767C'])
matplotlib.rcParams['axes.formatter.limits']=-3,3
# matplotlib.rcParams['axes.labelweight']=300

matplotlib.rcParams['legend.frameon']=False

matplotlib.rcParams['boxplot.whiskers']=(5,95)
matplotlib.rcParams['boxplot.showfliers']=False
matplotlib.rcParams['boxplot.showfliers']=False
matplotlib.rcParams['boxplot.medianprops.color']='black'

matplotlib.rcParams['errorbar.capsize']=5

matplotlib.rcParams['hist.bins']='auto'

plt.rcParams['pdf.fonttype'] = 42

%matplotlib inline

%load_ext line_profiler
%load_ext memory_profiler

The line_profiler extension is already loaded. To reload it, use:
  %reload_ext line_profiler
The memory_profiler extension is already loaded. To reload it, use:
  %reload_ext memory_profiler


## Load gas parameter sets + LUC scaling factors: assume no uncertainty from natural forcings?

In [4]:
default_params = pd.read_csv('../Parameter_Sets/Complete_gas_cycle_params.csv',skiprows=1,index_col=0)
forcing_namelist = list(default_params.columns)

param_categories = {}
# param_categories['other_wmghgs'] = ['c2f6', 'c3f8', 'c4f10', 'c5f12', 'c6f14', 'c7f16', 'c8f18', 'c_c4f8','carbon_tetrachloride', 'cf4', 'cfc113', 'cfc114', 'cfc115', 'cfc11', 'cfc12', 'ch2cl2', 'ch3ccl3', 'chcl3', 'halon1211', 'halon1301', 'halon2402', 'hcfc141b', 'hcfc142b', 'hcfc22', 'hfc125', 'hfc134a', 'hfc143a', 'hfc152a', 'hfc227ea', 'hfc236fa', 'hfc23', 'hfc245fa', 'hfc32', 'hfc365mfc', 'hfc4310mee', 'methyl_bromide', 'methyl_chloride', 'nf3', 'sf6', 'so2f2', 'halon1202']
param_categories['f_gases'] = ['c2f6', 'c3f8', 'c4f10', 'c5f12', 'c6f14', 'c7f16', 'c8f18', 'c_c4f8','nf3', 'sf6', 'so2f2','cf4','hfc125', 'hfc134a', 'hfc143a', 'hfc152a', 'hfc227ea', 'hfc236fa', 'hfc23', 'hfc245fa', 'hfc32', 'hfc365mfc', 'hfc4310mee']
param_categories['montreal_gases'] = ['carbon_tetrachloride','cfc113', 'cfc114', 'cfc115', 'cfc11', 'cfc12', 'ch2cl2', 'ch3ccl3', 'chcl3', 'halon1211', 'halon1301', 'halon2402', 'hcfc141b', 'hcfc142b', 'hcfc22','halon1202','methyl_bromide', 'methyl_chloride']
param_categories['carbon_dioxide'] = ['carbon_dioxide']
param_categories['methane'] = ['methane']
param_categories['nitrous_oxide'] = ['nitrous_oxide']
param_categories['trop_o3'] = [x for x in forcing_namelist if '|trop_o3' in x]
param_categories['strat_o3'] = [x for x in forcing_namelist if '|strat_o3' in x]
param_categories['strat_h2o'] = [x for x in forcing_namelist if '|strat_h2o' in x]
param_categories['ari'] = ['so2', 'nox', 'co', 'nmvoc', 'bc', 'nh3','oc','nox_avi']
param_categories['aci'] = ['so2|aci','oc|aci','bc|aci']
# param_categories['aer'] = param_categories['aci'] + param_categories['ari']
param_categories['bc_on_snow'] = ['bc|bc_on_snow']
param_categories['contrails'] = ['nox_avi|contrails']

In [5]:
basedir = '../Parameter_Sets/NROY/'
param_sets=[0,10000,20000,30000,40000,50000]

gas_param_ensemble = []
thermal_param_ensemble = []
EXT_forc_sfs = []

for param_set_start in param_sets:

    gas_param_file,response_param_file,level_constraint_file,rate_constrain_file,ext_forc_file = [basedir+x+'_mem'+str(param_set_start)+'-mem'+str(param_set_start+9999)+'_v1.pkl' for x in ['gas_params','response_params','temp_constraint','rate_constraint','EXT_forc']]
    level_constraint = pd.read_pickle(level_constraint_file)
    rate_constraint = pd.read_pickle(rate_constrain_file)
    full_constraint = level_constraint & rate_constraint
    NROY_mems = [x[0] for x in full_constraint[full_constraint].index]
    gas_param_ensemble += [pd.read_pickle(gas_param_file).reindex(NROY_mems,axis=1,level=0)]
    thermal_param_ensemble += [pd.read_pickle(response_param_file).reindex(NROY_mems,axis=1,level=0)]
    EXT_forc_sfs += [pd.read_pickle(ext_forc_file).reindex(NROY_mems,axis=1)]

thermal_param_ensemble = pd.concat(thermal_param_ensemble,axis=1)
gas_param_ensemble = pd.concat(gas_param_ensemble,axis=1)
EXT_forc_sfs = pd.concat(EXT_forc_sfs,axis=1)

In [6]:
## set the number of ensemble members desired:
N=7500

In [7]:
choose_mems = thermal_param_ensemble.columns.levels[0][:N]

thermal_param_ensemble = thermal_param_ensemble.reindex(choose_mems,axis=1,level=0)
gas_param_ensemble = gas_param_ensemble.reindex(choose_mems,axis=1,level=0)
EXT_forc_sfs = EXT_forc_sfs.reindex(choose_mems,axis=1)

### parameter set metrics:

In [6]:
member_metrics = pd.DataFrame(index=thermal_param_ensemble.columns.levels[0],columns=['Equilibrium Climate Sensitivity','Transient Climate Response'])

f_vals = gas_param_ensemble.loc['f1':'f3',(slice(None),'carbon_dioxide')].values
F_2x = f_vals[0] * np.log(2) + f_vals[2] * (np.sqrt(284*2)-np.sqrt(284))
member_metrics.loc[:,'Equilibrium Climate Sensitivity'] = F_2x*thermal_param_ensemble.loc['q'].values.reshape(-1,3).sum(axis=1)
member_metrics.loc[:,'Transient Climate Response'] = F_2x * ( thermal_param_ensemble.loc['q'].values.reshape(-1,3) * (1 - (thermal_param_ensemble.loc['d'].values.reshape(-1,3)/69.66) * ( 1 - np.exp(-69.66/thermal_param_ensemble.loc['d'].values.reshape(-1,3)) ) ) ).sum(axis=1)
member_metrics.loc[:,'F-2xCO2'] = F_2x

member_metrics = member_metrics.unstack().reset_index()
member_metrics.loc[:,'unit'] = 'K'
member_metrics.loc[:,'climate_model'] = 'FaIRv2.0.0-alpha'
member_metrics.columns = ['RCMIP name','ensemble_member','value','unit','climate_model']

In [7]:
member_metrics.to_hdf(r'/home/leachl/Documents/Simple_models/FaIR_v2-0/GIR/GIR_output/RCMIP/RCMIP_reported-metrics_FaIRv2.0.0-alpha.h5', key='stage', mode='w')

# Concentration-driven experiments

## Idealised experiments : 1pctCO2 , abrupt2xCO2, abrupt4xCO2, abrupt0.5xCO2

In [87]:
## create the scenarios:

idealised_concs = pd.DataFrame(284*1.01**(np.arange(200)),index=np.arange(200),columns=pd.MultiIndex.from_product([['1pctCO2'],['carbon_dioxide']]))
idealised_concs.loc[:,('abrupt-4xCO2','carbon_dioxide')] = 284*4
idealised_concs.loc[:,('abrupt-2xCO2','carbon_dioxide')] = 284*2
idealised_concs.loc[:,('abrupt-0p5xCO2','carbon_dioxide')] = 284/2
idealised_concs.loc[0,(slice(None),'carbon_dioxide')] = 284

idealised_forc = return_empty_forcing(idealised_concs)

idealised_params = gas_param_ensemble.reindex(['carbon_dioxide'],axis=1,level=1)
idealised_params.loc['PI_conc']=284

In [88]:
idealised_exp = run_GIR(concentrations_in=idealised_concs,forcing_in=idealised_forc,gas_parameters=idealised_params,thermal_parameters=thermal_param_ensemble)

Integrating 4 scenarios, 7500 gas cycle parameter sets, 1 thermal response parameter sets, over ['carbon_dioxide'] forcing agents, between 0 and 199...


100%|██████████| 199/199 [00:01<00:00, 99.71 timestep/s] 


In [89]:
T_out = idealised_exp['T'].loc[1:200].copy()

T_out = T_out.T.reset_index().rename({'Gas cycle set':'ensemble_member','Scenario':'scenario'},axis=1)

T_out.loc[:,'variable'] = 'Surface Air Temperature Change'
T_out.loc[:,'region'] = 'World'
T_out.loc[:,'climate_model'] = 'FaIRv2.0.0-alpha'
T_out.loc[:,'model'] = 'idealised'
T_out.loc[:,'unit'] = 'K'

T_out = T_out.set_index(['climate_model','model','scenario','region','variable','unit','ensemble_member']).reset_index()

In [90]:
# get result category intersection with all possible categories:

RF_out = []

exp_forc_cats = [x for x,vals in param_categories.items() if bool(set(vals).intersection(idealised_exp['RF'].columns.levels[-1]))]

for forc_cat in exp_forc_cats:
#     RF_out.loc[:,(slice(None),slice(None),forc_cat)] = 
    RF_out += [idealised_exp['RF'].loc[:,(slice(None),slice(None),param_categories[forc_cat])].groupby(level=[0,1],axis=1).sum()]
    
RF_out += [idealised_exp['RF'].xs('Total',axis=1,level=-1)]

RF_out = pd.concat(RF_out,axis=1,keys=exp_forc_cats+['Effective Radiative Forcing'])
RF_out = RF_out.loc[1:200].T.reset_index().rename({'Gas cycle set':'ensemble_member','Scenario':'scenario','level_0':'variable'},axis=1)

for forc_cat in exp_forc_cats:
    RF_out.replace(forc_cat,GIR_to_RCMIP_map_forc.loc[forc_cat,'RCMIP_forc_key'],inplace=True)
    
RF_out.loc[:,'region'] = 'World'
RF_out.loc[:,'climate_model'] = 'FaIRv2.0.0-alpha'
RF_out.loc[:,'model'] = 'idealised'
RF_out.loc[:,'unit'] = 'W/m^2'

RF_out = RF_out.set_index(['climate_model','model','scenario','region','variable','unit','ensemble_member']).reset_index()

In [91]:
emms_out = idealised_exp['Emissions'].apply(lambda x: x/GIR_to_RCMIP_map.loc[x.name[-1],'RCMIP_emms_scaling'],raw=False).copy()

emms_out = emms_out.T.reset_index().rename({'Gas cycle set':'ensemble_member','Scenario':'scenario','Gas name':'variable'},axis=1)

exp_emms_cats = [x for x in GIR_to_RCMIP_map.index if x in idealised_exp['Emissions'].columns.levels[-1]]

for agent in exp_emms_cats:
    emms_out.replace(agent,GIR_to_RCMIP_map.loc[agent,'RCMIP_emms_key'],inplace=True)
    
emms_out.loc[:,'region'] = 'World'
emms_out.loc[:,'climate_model'] = 'FaIRv2.0.0-alpha'
emms_out.loc[:,'model'] = 'idealised'
emms_out.loc[:,'unit'] = GIR_to_RCMIP_map.set_index('RCMIP_emms_key').loc[emms_out.variable,'RCMIP_emms_unit'].values

emms_out = emms_out.set_index(['climate_model','model','scenario','region','variable','unit','ensemble_member']).reset_index()

In [105]:
cum_emms_out = emms_out.loc[emms_out.variable=='Emissions|CO2'].copy()
atmos_pool_out = emms_out.loc[emms_out.variable=='Emissions|CO2'].copy()
af_out = emms_out.loc[emms_out.variable=='Emissions|CO2'].copy()
TCRE_out = emms_out.loc[emms_out.variable=='Emissions|CO2'].copy()

cum_emms_out = cum_emms_out.set_index(['climate_model','model','scenario','region','variable','unit','ensemble_member']).cumsum(axis=1).reset_index()
cum_emms_out.loc[:,'variable'] = 'Cumulative Emissions|CO2'

atmos_pool_out = atmos_pool_out.set_index(['climate_model','model','scenario','region','variable','unit','ensemble_member']).apply(lambda x: (idealised_exp['C'].loc[:,(x.name[2],'carbon_dioxide')]-284)/(4.688876e-01*GIR_to_RCMIP_map.loc['carbon_dioxide','RCMIP_emms_scaling']),axis=1).reset_index()
atmos_pool_out.loc[:,'variable'] = 'Carbon Pool|Atmosphere'
atmos_pool_out.loc[:,'unit'] = 'Mt CO2'

af_out = (atmos_pool_out.set_index(['climate_model','model','scenario','region','ensemble_member']).iloc[:,2:] / cum_emms_out.set_index(['climate_model','model','scenario','region','ensemble_member']).iloc[:,2:]).reset_index()
af_out.loc[:,'variable'] = 'Airborne Fraction|CO2'
af_out.loc[:,'unit'] = '-'
af_out = af_out.set_index(['climate_model','model','scenario','region','variable','unit','ensemble_member']).reset_index()

TCRE_out = (T_out.set_index(['climate_model','model','scenario','region','ensemble_member']).iloc[:,2:] / cum_emms_out.set_index(['climate_model','model','scenario','region','ensemble_member']).iloc[:,2:]).reset_index()
TCRE_out.loc[:,'variable'] = 'Instantaneous TCRE'
TCRE_out.loc[:,'unit'] = 'K / MtCO2'
TCRE_out = TCRE_out.set_index(['climate_model','model','scenario','region','variable','unit','ensemble_member']).reset_index()

In [106]:
output = pd.concat([T_out,RF_out,emms_out,cum_emms_out,atmos_pool_out,af_out,TCRE_out],axis=0).dropna(axis=1)

output.to_hdf(r'/home/leachl/Documents/Simple_models/FaIR_v2-0/GIR/GIR_output/RCMIP/RCMIP_idealised_FaIRv2.0.0-alpha.h5', key='stage', mode='w')

/home/leachl/miniconda3/lib/python3.7/site-packages/pandas/io/pytables.py:282: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types that it cannot
map directly to c-types [inferred_type->mixed-integer,key->axis0] [items->None]

  f(store)
/home/leachl/miniconda3/lib/python3.7/site-packages/pandas/io/pytables.py:282: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types that it cannot
map directly to c-types [inferred_type->integer,key->block1_items] [items->None]

  f(store)


## SSPs 

In [8]:
def run_SSP_RCMIP(ssp,save_period=range(2014,2301)):

    SSP_concs = RCMIP_to_GIR_input_concs(ssp)
    SSP_emms = RCMIP_to_GIR_input_emms(ssp)

    aer_species = [x for x in SSP_concs.columns if SSP_concs[x].dropna().empty]

    SSP_concs.loc[:,aer_species] = SSP_emms.loc[:,aer_species] - SSP_emms.loc[1750,aer_species]

    SSP_concs = SSP_concs.loc[1750:2300].interpolate().dropna(how='all')

    SSP_LUC = pd.concat([get_RCMIP_forc([ssp],'Effective Radiative Forcing|Anthropogenic|Albedo Change')]*N,axis=1,keys=EXT_forc_sfs.loc['LUC'].index)*EXT_forc_sfs.loc['LUC'].values

    SSP_other_forc = get_RCMIP_forc([ssp],['Effective Radiative Forcing|Natural'])

    SSP_forc = (SSP_LUC + SSP_other_forc.values).loc[SSP_concs.index]

    SSP_concs = pd.concat([SSP_concs]*N,axis=1,keys=EXT_forc_sfs.loc['LUC'].index)

    ## run the model:
    SSP_run = run_GIR(concentrations_in=SSP_concs,forcing_in=SSP_forc,gas_parameters=gas_param_ensemble,thermal_parameters=thermal_param_ensemble,aer_concs_in=aer_species)

    ## convert output to RCMIP formatting reqs:
    T_out = pd.concat([SSP_run['T'].loc[:].copy()],axis=1,keys=[ssp],names=['Scenario','Gas cycle set'])
    T_out = T_out.T.reset_index().rename({'Gas cycle set':'ensemble_member','Scenario':'scenario'},axis=1)
    T_out.loc[:,'variable'] = 'Surface Air Temperature Change'
    T_out.loc[:,'region'] = 'World'
    T_out.loc[:,'climate_model'] = 'FaIRv2.0.0-alpha'
    T_out.loc[:,'model'] = RCMIP_concs.loc[('World',ssp)].Model.unique()[0]
    T_out.loc[:,'unit'] = 'K'
    T_out = T_out.set_index(['climate_model','model','scenario','region','variable','unit','ensemble_member']).reset_index()

    # get result category intersection with all possible categories:

    RF_out = []
    exp_forc_cats = [x for x,vals in param_categories.items() if bool(set(vals).intersection(SSP_run['RF'].columns.levels[-1]))]
    for forc_cat in exp_forc_cats:
    #     RF_out.loc[:,(slice(None),slice(None),forc_cat)] = 
        RF_out += [SSP_run['RF'].loc[:,(slice(None),param_categories[forc_cat])].groupby(level=0,axis=1).sum()]
    RF_out += [SSP_run['RF'].loc[:].xs('Total',axis=1,level=-1)]
    RF_out = pd.concat(RF_out,axis=1,keys=exp_forc_cats+['Effective Radiative Forcing'])
    RF_out = pd.concat([RF_out],axis=1,keys=[ssp],names=['scenario','variable','ensemble_member'])
    RF_out = RF_out.T.reset_index()
    for forc_cat in exp_forc_cats:
        RF_out.replace(forc_cat,GIR_to_RCMIP_map_forc.loc[forc_cat,'RCMIP_forc_key'],inplace=True)
    RF_out.loc[:,'region'] = 'World'
    RF_out.loc[:,'climate_model'] = 'FaIRv2.0.0-alpha'
    RF_out.loc[:,'model'] = RCMIP_concs.loc[('World',ssp)].Model.unique()[0]
    RF_out.loc[:,'unit'] = 'W/m^2'
    RF_out = RF_out.set_index(['climate_model','model','scenario','region','variable','unit','ensemble_member']).reset_index()

    emms_out = SSP_run['Emissions'].loc[:,(slice(None),'carbon_dioxide')].apply(lambda x: x/GIR_to_RCMIP_map.loc[x.name[-1],'RCMIP_emms_scaling'],raw=False).copy()
    emms_out = pd.concat([emms_out],axis=1,keys=[ssp],names=['scenario','ensemble_member','variable']).T.reset_index()
    exp_emms_cats = [x for x in GIR_to_RCMIP_map.index if x in emms_out.variable.unique()]
    for agent in exp_emms_cats:
        emms_out.replace(agent,GIR_to_RCMIP_map.loc[agent,'RCMIP_emms_key'],inplace=True)
    emms_out.loc[:,'region'] = 'World'
    emms_out.loc[:,'climate_model'] = 'FaIRv2.0.0-alpha'
    emms_out.loc[:,'model'] = RCMIP_concs.loc[('World',ssp)].Model.unique()[0]
    emms_out.loc[:,'unit'] = GIR_to_RCMIP_map.set_index('RCMIP_emms_key').loc[emms_out.variable,'RCMIP_emms_unit'].values
    emms_out = emms_out.set_index(['climate_model','model','scenario','region','variable','unit','ensemble_member']).reset_index()

    cum_emms_out = emms_out.loc[emms_out.variable=='Emissions|CO2'].copy()
    atmos_pool_out = emms_out.loc[emms_out.variable=='Emissions|CO2'].copy()
    af_out = emms_out.loc[emms_out.variable=='Emissions|CO2'].copy()
    TCRE_out = emms_out.loc[emms_out.variable=='Emissions|CO2'].copy()

    cum_emms_out = cum_emms_out.set_index(['climate_model','model','scenario','region','variable','unit','ensemble_member']).cumsum(axis=1).reset_index()
    cum_emms_out.loc[:,'variable'] = 'Cumulative Emissions|CO2'

    atmos_pool_out = atmos_pool_out.set_index(['climate_model','model','scenario','region','variable','unit','ensemble_member']).apply(lambda x: (SSP_run['C'].loc[:,(x.name[-1],'carbon_dioxide')]-278)/(4.688876e-01*GIR_to_RCMIP_map.loc['carbon_dioxide','RCMIP_emms_scaling']),axis=1).reset_index()
    atmos_pool_out.loc[:,'variable'] = 'Carbon Pool|Atmosphere'
    atmos_pool_out.loc[:,'unit'] = 'Mt CO2'

    af_out = (atmos_pool_out.set_index(['climate_model','model','scenario','region','ensemble_member']).iloc[:,2:] / cum_emms_out.set_index(['climate_model','model','scenario','region','ensemble_member']).iloc[:,2:]).reset_index()
    af_out.loc[:,'variable'] = 'Airborne Fraction|CO2'
    af_out.loc[:,'unit'] = '-'
    af_out = af_out.set_index(['climate_model','model','scenario','region','variable','unit','ensemble_member']).reset_index()

    TCRE_out = (T_out.set_index(['climate_model','model','scenario','region','ensemble_member']).iloc[:,2:] / cum_emms_out.set_index(['climate_model','model','scenario','region','ensemble_member']).iloc[:,2:]).reset_index()
    TCRE_out.loc[:,'variable'] = 'Instantaneous TCRE'
    TCRE_out.loc[:,'unit'] = 'K / MtCO2'
    TCRE_out = TCRE_out.set_index(['climate_model','model','scenario','region','variable','unit','ensemble_member']).reset_index()

    CH4_tau = pd.concat([pd.concat([SSP_run['alpha'].xs('methane',axis=1,level=-1)*gas_param_ensemble.loc['tau1',(slice(None),'methane')].unique()[0]],axis=1,keys=['Atmospheric Lifetime|CH4'])],axis=1,keys=[ssp],names=['scenario','variable','ensemble_member'])
    CH4_tau = CH4_tau.T.reset_index()
    CH4_tau.loc[:,'region'] = 'World'
    CH4_tau.loc[:,'climate_model'] = 'FaIRv2.0.0-alpha'
    CH4_tau.loc[:,'model'] = RCMIP_concs.loc[('World',ssp)].Model.unique()[0]
    CH4_tau.loc[:,'unit'] = 'yr'
    CH4_tau = CH4_tau.set_index(['climate_model','model','scenario','region','variable','unit','ensemble_member']).reset_index()

    output = pd.concat([T_out,RF_out,emms_out,cum_emms_out,atmos_pool_out,af_out,TCRE_out,CH4_tau],axis=0).loc[:,['climate_model','model','scenario','region','variable','unit','ensemble_member']+list(save_period)]

    output.to_hdf(r'/home/leachl/Documents/Simple_models/FaIR_v2-0/GIR/GIR_output/RCMIP/RCMIP_'+ssp+'_FaIRv2.0.0-alpha.h5', key='stage', mode='w')

In [9]:
for ssp in [x for x in RCMIP_concs.index.levels[-2] if 'ssp' in x]:
    run_SSP_RCMIP(ssp,range(1850,2301))
    
# run_SSP_RCMIP('historical',range(1750,2015))

Integrating 7500 scenarios, 1 gas cycle parameter sets, 1 thermal response parameter sets, over ['bc', 'bc|aci', 'bc|bc_on_snow', 'c2f6', 'c3f8', 'c4f10', 'c5f12', 'c6f14', 'c7f16', 'c8f18', 'c_c4f8', 'carbon_dioxide', 'carbon_tetrachloride', 'carbon_tetrachloride|strat_o3', 'cf4', 'cfc11', 'cfc113', 'cfc113|strat_o3', 'cfc114', 'cfc114|strat_o3', 'cfc115', 'cfc115|strat_o3', 'cfc11|strat_o3', 'cfc12', 'cfc12|strat_o3', 'ch2cl2', 'ch3ccl3', 'ch3ccl3|strat_o3', 'chcl3', 'co', 'co|trop_o3', 'halon1202', 'halon1202|strat_o3', 'halon1211', 'halon1211|strat_o3', 'halon1301', 'halon1301|strat_o3', 'halon2402', 'halon2402|strat_o3', 'hcfc141b', 'hcfc141b|strat_o3', 'hcfc142b', 'hcfc142b|strat_o3', 'hcfc22', 'hcfc22|strat_o3', 'hfc125', 'hfc134a', 'hfc143a', 'hfc152a', 'hfc227ea', 'hfc23', 'hfc236fa', 'hfc245fa', 'hfc32', 'hfc365mfc', 'hfc4310mee', 'methane', 'methane|strat_h2o', 'methane|trop_o3', 'methyl_bromide', 'methyl_bromide|strat_o3', 'methyl_chloride', 'methyl_chloride|strat_o3', 'nf3

100%|██████████| 550/550 [01:43<00:00,  5.34 timestep/s]
/home/leachl/miniconda3/lib/python3.7/site-packages/ipykernel_launcher.py:29: PerformanceWarning: indexing past lexsort depth may impact performance.
/home/leachl/miniconda3/lib/python3.7/site-packages/ipykernel_launcher.py:48: PerformanceWarning: indexing past lexsort depth may impact performance.
/home/leachl/miniconda3/lib/python3.7/site-packages/ipykernel_launcher.py:59: PerformanceWarning: indexing past lexsort depth may impact performance.
/home/leachl/miniconda3/lib/python3.7/site-packages/ipykernel_launcher.py:89: PerformanceWarning: indexing past lexsort depth may impact performance.
/home/leachl/miniconda3/lib/python3.7/site-packages/pandas/io/pytables.py:282: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types that it cannot
map directly to c-types [inferred_type->mixed-integer,key->axis0] [items->None]

  f(store)
/home/leachl/miniconda3/lib/python3.7/site-packages/pandas/io/pytables.

Integrating 7500 scenarios, 1 gas cycle parameter sets, 1 thermal response parameter sets, over ['bc', 'bc|aci', 'bc|bc_on_snow', 'c2f6', 'c3f8', 'c4f10', 'c5f12', 'c6f14', 'c7f16', 'c8f18', 'c_c4f8', 'carbon_dioxide', 'carbon_tetrachloride', 'carbon_tetrachloride|strat_o3', 'cf4', 'cfc11', 'cfc113', 'cfc113|strat_o3', 'cfc114', 'cfc114|strat_o3', 'cfc115', 'cfc115|strat_o3', 'cfc11|strat_o3', 'cfc12', 'cfc12|strat_o3', 'ch2cl2', 'ch3ccl3', 'ch3ccl3|strat_o3', 'chcl3', 'co', 'co|trop_o3', 'halon1202', 'halon1202|strat_o3', 'halon1211', 'halon1211|strat_o3', 'halon1301', 'halon1301|strat_o3', 'halon2402', 'halon2402|strat_o3', 'hcfc141b', 'hcfc141b|strat_o3', 'hcfc142b', 'hcfc142b|strat_o3', 'hcfc22', 'hcfc22|strat_o3', 'hfc125', 'hfc134a', 'hfc143a', 'hfc152a', 'hfc227ea', 'hfc23', 'hfc236fa', 'hfc245fa', 'hfc32', 'hfc365mfc', 'hfc4310mee', 'methane', 'methane|strat_h2o', 'methane|trop_o3', 'methyl_bromide', 'methyl_bromide|strat_o3', 'methyl_chloride', 'methyl_chloride|strat_o3', 'nf3

100%|██████████| 550/550 [01:42<00:00,  5.37 timestep/s]


Integrating 7500 scenarios, 1 gas cycle parameter sets, 1 thermal response parameter sets, over ['bc', 'bc|aci', 'bc|bc_on_snow', 'c2f6', 'c3f8', 'c4f10', 'c5f12', 'c6f14', 'c7f16', 'c8f18', 'c_c4f8', 'carbon_dioxide', 'carbon_tetrachloride', 'carbon_tetrachloride|strat_o3', 'cf4', 'cfc11', 'cfc113', 'cfc113|strat_o3', 'cfc114', 'cfc114|strat_o3', 'cfc115', 'cfc115|strat_o3', 'cfc11|strat_o3', 'cfc12', 'cfc12|strat_o3', 'ch2cl2', 'ch3ccl3', 'ch3ccl3|strat_o3', 'chcl3', 'co', 'co|trop_o3', 'halon1202', 'halon1202|strat_o3', 'halon1211', 'halon1211|strat_o3', 'halon1301', 'halon1301|strat_o3', 'halon2402', 'halon2402|strat_o3', 'hcfc141b', 'hcfc141b|strat_o3', 'hcfc142b', 'hcfc142b|strat_o3', 'hcfc22', 'hcfc22|strat_o3', 'hfc125', 'hfc134a', 'hfc143a', 'hfc152a', 'hfc227ea', 'hfc23', 'hfc236fa', 'hfc245fa', 'hfc32', 'hfc365mfc', 'hfc4310mee', 'methane', 'methane|strat_h2o', 'methane|trop_o3', 'methyl_bromide', 'methyl_bromide|strat_o3', 'methyl_chloride', 'methyl_chloride|strat_o3', 'nf3

...

100%|██████████| 550/550 [01:40<00:00,  5.45 timestep/s]


Integrating 7500 scenarios, 1 gas cycle parameter sets, 1 thermal response parameter sets, over ['bc', 'bc|aci', 'bc|bc_on_snow', 'c2f6', 'c3f8', 'c4f10', 'c5f12', 'c6f14', 'c7f16', 'c8f18', 'c_c4f8', 'carbon_dioxide', 'carbon_tetrachloride', 'carbon_tetrachloride|strat_o3', 'cf4', 'cfc11', 'cfc113', 'cfc113|strat_o3', 'cfc114', 'cfc114|strat_o3', 'cfc115', 'cfc115|strat_o3', 'cfc11|strat_o3', 'cfc12', 'cfc12|strat_o3', 'ch2cl2', 'ch3ccl3', 'ch3ccl3|strat_o3', 'chcl3', 'co', 'co|trop_o3', 'halon1202', 'halon1202|strat_o3', 'halon1211', 'halon1211|strat_o3', 'halon1301', 'halon1301|strat_o3', 'halon2402', 'halon2402|strat_o3', 'hcfc141b', 'hcfc141b|strat_o3', 'hcfc142b', 'hcfc142b|strat_o3', 'hcfc22', 'hcfc22|strat_o3', 'hfc125', 'hfc134a', 'hfc143a', 'hfc152a', 'hfc227ea', 'hfc23', 'hfc236fa', 'hfc245fa', 'hfc32', 'hfc365mfc', 'hfc4310mee', 'methane', 'methane|strat_h2o', 'methane|trop_o3', 'methyl_bromide', 'methyl_bromide|strat_o3', 'methyl_chloride', 'methyl_chloride|strat_o3', 'nf3

100%|██████████| 550/550 [01:47<00:00,  5.11 timestep/s]


Integrating 7500 scenarios, 1 gas cycle parameter sets, 1 thermal response parameter sets, over ['bc', 'bc|aci', 'bc|bc_on_snow', 'c2f6', 'c3f8', 'c4f10', 'c5f12', 'c6f14', 'c7f16', 'c8f18', 'c_c4f8', 'carbon_dioxide', 'carbon_tetrachloride', 'carbon_tetrachloride|strat_o3', 'cf4', 'cfc11', 'cfc113', 'cfc113|strat_o3', 'cfc114', 'cfc114|strat_o3', 'cfc115', 'cfc115|strat_o3', 'cfc11|strat_o3', 'cfc12', 'cfc12|strat_o3', 'ch2cl2', 'ch3ccl3', 'ch3ccl3|strat_o3', 'chcl3', 'co', 'co|trop_o3', 'halon1202', 'halon1202|strat_o3', 'halon1211', 'halon1211|strat_o3', 'halon1301', 'halon1301|strat_o3', 'halon2402', 'halon2402|strat_o3', 'hcfc141b', 'hcfc141b|strat_o3', 'hcfc142b', 'hcfc142b|strat_o3', 'hcfc22', 'hcfc22|strat_o3', 'hfc125', 'hfc134a', 'hfc143a', 'hfc152a', 'hfc227ea', 'hfc23', 'hfc236fa', 'hfc245fa', 'hfc32', 'hfc365mfc', 'hfc4310mee', 'methane', 'methane|strat_h2o', 'methane|trop_o3', 'methyl_bromide', 'methyl_bromide|strat_o3', 'methyl_chloride', 'methyl_chloride|strat_o3', 'nf3

100%|██████████| 550/550 [01:44<00:00,  5.24 timestep/s]


## RCPs

In [11]:
def run_RCP(rcp,save_period=range(2004,2301)):

    RCP_concs = RCMIP_to_GIR_input_concs(rcp)
    RCP_emms = RCMIP_to_GIR_input_emms(rcp)

    aer_species = [x for x in RCP_concs.columns if RCP_concs[x].dropna().empty]

    aer_species = RCP_emms.loc[:,aer_species].dropna(axis=1,how='all').columns

    RCP_concs.loc[:,aer_species] = RCP_emms.loc[:,aer_species] - RCP_emms.loc[1765,aer_species]

    RCP_concs = RCP_concs.loc[1750:2300].interpolate(limit_area='inside').dropna(how='all').dropna(how='all',axis=1)

    RCP_LUC = pd.concat([get_RCMIP_forc([rcp],'Radiative Forcing|Anthropogenic|Albedo Change')]*N,axis=1,keys=EXT_forc_sfs.loc['LUC'].index)*EXT_forc_sfs.loc['LUC'].values

    RCP_other_forc = get_RCMIP_forc([rcp],['Radiative Forcing|Natural'])

    RCP_forc = (RCP_LUC + RCP_other_forc.values).loc[:2300].dropna(how='all')

    RCP_species = RCP_concs.columns.unique()

    RCP_concs = pd.concat([RCP_concs]*N,axis=1,keys=EXT_forc_sfs.loc['LUC'].index)

    RCP_gas_params = gas_param_ensemble.reindex([x for x in gas_param_ensemble.columns.levels[-1] if x.split('|')[0] in RCP_species],axis=1,level=-1)

    ## run the model:
    RCP_run = run_GIR(concentrations_in=RCP_concs,forcing_in=RCP_forc,gas_parameters=RCP_gas_params,thermal_parameters=thermal_param_ensemble,aer_concs_in=aer_species)

## convert output to RCMIP formatting reqs:
    T_out = pd.concat([RCP_run['T'].loc[:].copy()],axis=1,keys=[rcp],names=['Scenario','Gas cycle set'])
    T_out = T_out.T.reset_index().rename({'Gas cycle set':'ensemble_member','Scenario':'scenario'},axis=1)
    T_out.loc[:,'variable'] = 'Surface Air Temperature Change'
    T_out.loc[:,'region'] = 'World'
    T_out.loc[:,'climate_model'] = 'FaIRv2.0.0-alpha'
    T_out.loc[:,'model'] = RCMIP_concs.loc[('World',rcp)].Model.unique()[0]
    T_out.loc[:,'unit'] = 'K'
    T_out = T_out.set_index(['climate_model','model','scenario','region','variable','unit','ensemble_member']).reset_index()

    # get result category intersection with all possible categories:

    RF_out = []
    exp_forc_cats = [x for x,vals in param_categories.items() if bool(set(vals).intersection(RCP_run['RF'].columns.levels[-1]))]
    for forc_cat in exp_forc_cats:
        forc_cat_intersect = set(param_categories[forc_cat]).intersection(set(RCP_run['RF'].columns.levels[-1]))
        RF_out += [RCP_run['RF'].loc[:,(slice(None),forc_cat_intersect)].groupby(level=0,axis=1).sum()]
    RF_out += [RCP_run['RF'].loc[:].xs('Total',axis=1,level=-1)]
    RF_out = pd.concat(RF_out,axis=1,keys=exp_forc_cats+['Effective Radiative Forcing'])
    RF_out = pd.concat([RF_out],axis=1,keys=[rcp],names=['scenario','variable','ensemble_member'])
    RF_out = RF_out.T.reset_index()
    for forc_cat in exp_forc_cats:
        RF_out.replace(forc_cat,GIR_to_RCMIP_map_forc.loc[forc_cat,'RCMIP_forc_key'],inplace=True)
    RF_out.loc[:,'region'] = 'World'
    RF_out.loc[:,'climate_model'] = 'FaIRv2.0.0-alpha'
    RF_out.loc[:,'model'] = RCMIP_concs.loc[('World',rcp)].Model.unique()[0]
    RF_out.loc[:,'unit'] = 'W/m^2'
    RF_out = RF_out.set_index(['climate_model','model','scenario','region','variable','unit','ensemble_member']).reset_index()

    emms_out = RCP_run['Emissions'].loc[:,(slice(None),'carbon_dioxide')].apply(lambda x: x/GIR_to_RCMIP_map.loc[x.name[-1],'RCMIP_emms_scaling'],raw=False).copy()
    emms_out = pd.concat([emms_out],axis=1,keys=[rcp],names=['scenario','ensemble_member','variable']).T.reset_index()
    exp_emms_cats = [x for x in GIR_to_RCMIP_map.index if x in emms_out.variable.unique()]
    for agent in exp_emms_cats:
        emms_out.replace(agent,GIR_to_RCMIP_map.loc[agent,'RCMIP_emms_key'],inplace=True)
    emms_out.loc[:,'region'] = 'World'
    emms_out.loc[:,'climate_model'] = 'FaIRv2.0.0-alpha'
    emms_out.loc[:,'model'] = RCMIP_concs.loc[('World',rcp)].Model.unique()[0]
    emms_out.loc[:,'unit'] = GIR_to_RCMIP_map.set_index('RCMIP_emms_key').loc[emms_out.variable,'RCMIP_emms_unit'].values
    emms_out = emms_out.set_index(['climate_model','model','scenario','region','variable','unit','ensemble_member']).reset_index()

    cum_emms_out = emms_out.loc[emms_out.variable=='Emissions|CO2'].copy()
    atmos_pool_out = emms_out.loc[emms_out.variable=='Emissions|CO2'].copy()
    af_out = emms_out.loc[emms_out.variable=='Emissions|CO2'].copy()
    TCRE_out = emms_out.loc[emms_out.variable=='Emissions|CO2'].copy()

    cum_emms_out = cum_emms_out.set_index(['climate_model','model','scenario','region','variable','unit','ensemble_member']).cumsum(axis=1).reset_index()
    cum_emms_out.loc[:,'variable'] = 'Cumulative Emissions|CO2'

    atmos_pool_out = atmos_pool_out.set_index(['climate_model','model','scenario','region','variable','unit','ensemble_member']).apply(lambda x: (RCP_run['C'].loc[:,(x.name[-1],'carbon_dioxide')]-278)/(4.688876e-01*GIR_to_RCMIP_map.loc['carbon_dioxide','RCMIP_emms_scaling']),axis=1).reset_index()
    atmos_pool_out.loc[:,'variable'] = 'Carbon Pool|Atmosphere'
    atmos_pool_out.loc[:,'unit'] = 'Mt CO2'

    af_out = (atmos_pool_out.set_index(['climate_model','model','scenario','region','ensemble_member']).iloc[:,2:] / cum_emms_out.set_index(['climate_model','model','scenario','region','ensemble_member']).iloc[:,2:]).reset_index()
    af_out.loc[:,'variable'] = 'Airborne Fraction|CO2'
    af_out.loc[:,'unit'] = '-'
    af_out = af_out.set_index(['climate_model','model','scenario','region','variable','unit','ensemble_member']).reset_index()

    TCRE_out = (T_out.set_index(['climate_model','model','scenario','region','ensemble_member']).iloc[:,2:] / cum_emms_out.set_index(['climate_model','model','scenario','region','ensemble_member']).iloc[:,2:]).reset_index()
    TCRE_out.loc[:,'variable'] = 'Instantaneous TCRE'
    TCRE_out.loc[:,'unit'] = 'K / MtCO2'
    TCRE_out = TCRE_out.set_index(['climate_model','model','scenario','region','variable','unit','ensemble_member']).reset_index()

    CH4_tau = pd.concat([pd.concat([RCP_run['alpha'].xs('methane',axis=1,level=-1)*gas_param_ensemble.loc['tau1',(slice(None),'methane')].unique()[0]],axis=1,keys=['Atmospheric Lifetime|CH4'])],axis=1,keys=[rcp],names=['scenario','variable','ensemble_member'])
    CH4_tau = CH4_tau.T.reset_index()
    CH4_tau.loc[:,'region'] = 'World'
    CH4_tau.loc[:,'climate_model'] = 'FaIRv2.0.0-alpha'
    CH4_tau.loc[:,'model'] = RCMIP_concs.loc[('World',rcp)].Model.unique()[0]
    CH4_tau.loc[:,'unit'] = 'yr'
    CH4_tau = CH4_tau.set_index(['climate_model','model','scenario','region','variable','unit','ensemble_member']).reset_index()

    output = pd.concat([T_out,RF_out,emms_out,cum_emms_out,atmos_pool_out,af_out,TCRE_out,CH4_tau],axis=0).loc[:,['climate_model','model','scenario','region','variable','unit','ensemble_member']+list(save_period)]

    output.to_hdf(r'/home/leachl/Documents/Simple_models/FaIR_v2-0/GIR/GIR_output/RCMIP/RCMIP_'+rcp+'_FaIRv2.0.0-alpha.h5', key='stage', mode='w')

In [12]:
for rcp in [x for x in RCMIP_concs.index.levels[-2] if 'rcp' in x]:
    run_RCP(rcp,range(1850,2301))
    
# run_RCP('historical-cmip5',range(1765,2005))

Integrating 7500 scenarios, 1 gas cycle parameter sets, 1 thermal response parameter sets, over ['bc', 'bc|aci', 'bc|bc_on_snow', 'c2f6', 'c6f14', 'carbon_dioxide', 'carbon_tetrachloride', 'carbon_tetrachloride|strat_o3', 'cf4', 'cfc11', 'cfc113', 'cfc113|strat_o3', 'cfc114', 'cfc114|strat_o3', 'cfc115', 'cfc115|strat_o3', 'cfc11|strat_o3', 'cfc12', 'cfc12|strat_o3', 'ch3ccl3', 'ch3ccl3|strat_o3', 'co', 'co|trop_o3', 'halon1202', 'halon1202|strat_o3', 'halon1211', 'halon1211|strat_o3', 'halon1301', 'halon1301|strat_o3', 'halon2402', 'halon2402|strat_o3', 'hcfc141b', 'hcfc141b|strat_o3', 'hcfc142b', 'hcfc142b|strat_o3', 'hcfc22', 'hcfc22|strat_o3', 'hfc125', 'hfc134a', 'hfc143a', 'hfc227ea', 'hfc23', 'hfc245fa', 'hfc32', 'hfc4310mee', 'methane', 'methane|strat_h2o', 'methane|trop_o3', 'methyl_bromide', 'methyl_bromide|strat_o3', 'methyl_chloride', 'methyl_chloride|strat_o3', 'nh3', 'nitrous_oxide', 'nmvoc', 'nmvoc|trop_o3', 'nox', 'nox|trop_o3', 'oc', 'oc|aci', 'sf6', 'so2', 'so2|aci'] 

100%|██████████| 535/535 [01:11<00:00,  7.46 timestep/s]
/home/leachl/miniconda3/lib/python3.7/site-packages/ipykernel_launcher.py:35: PerformanceWarning: indexing past lexsort depth may impact performance.
/home/leachl/miniconda3/lib/python3.7/site-packages/ipykernel_launcher.py:54: PerformanceWarning: indexing past lexsort depth may impact performance.
/home/leachl/miniconda3/lib/python3.7/site-packages/ipykernel_launcher.py:65: PerformanceWarning: indexing past lexsort depth may impact performance.
/home/leachl/miniconda3/lib/python3.7/site-packages/ipykernel_launcher.py:95: PerformanceWarning: indexing past lexsort depth may impact performance.


Integrating 7500 scenarios, 1 gas cycle parameter sets, 1 thermal response parameter sets, over ['bc', 'bc|aci', 'bc|bc_on_snow', 'c2f6', 'c6f14', 'carbon_dioxide', 'carbon_tetrachloride', 'carbon_tetrachloride|strat_o3', 'cf4', 'cfc11', 'cfc113', 'cfc113|strat_o3', 'cfc114', 'cfc114|strat_o3', 'cfc115', 'cfc115|strat_o3', 'cfc11|strat_o3', 'cfc12', 'cfc12|strat_o3', 'ch3ccl3', 'ch3ccl3|strat_o3', 'co', 'co|trop_o3', 'halon1202', 'halon1202|strat_o3', 'halon1211', 'halon1211|strat_o3', 'halon1301', 'halon1301|strat_o3', 'halon2402', 'halon2402|strat_o3', 'hcfc141b', 'hcfc141b|strat_o3', 'hcfc142b', 'hcfc142b|strat_o3', 'hcfc22', 'hcfc22|strat_o3', 'hfc125', 'hfc134a', 'hfc143a', 'hfc227ea', 'hfc23', 'hfc245fa', 'hfc32', 'hfc4310mee', 'methane', 'methane|strat_h2o', 'methane|trop_o3', 'methyl_bromide', 'methyl_bromide|strat_o3', 'methyl_chloride', 'methyl_chloride|strat_o3', 'nh3', 'nitrous_oxide', 'nmvoc', 'nmvoc|trop_o3', 'nox', 'nox|trop_o3', 'oc', 'oc|aci', 'sf6', 'so2', 'so2|aci'] 

100%|██████████| 535/535 [01:12<00:00,  7.39 timestep/s]


Integrating 7500 scenarios, 1 gas cycle parameter sets, 1 thermal response parameter sets, over ['bc', 'bc|aci', 'bc|bc_on_snow', 'c2f6', 'c6f14', 'carbon_dioxide', 'carbon_tetrachloride', 'carbon_tetrachloride|strat_o3', 'cf4', 'cfc11', 'cfc113', 'cfc113|strat_o3', 'cfc114', 'cfc114|strat_o3', 'cfc115', 'cfc115|strat_o3', 'cfc11|strat_o3', 'cfc12', 'cfc12|strat_o3', 'ch3ccl3', 'ch3ccl3|strat_o3', 'co', 'co|trop_o3', 'halon1202', 'halon1202|strat_o3', 'halon1211', 'halon1211|strat_o3', 'halon1301', 'halon1301|strat_o3', 'halon2402', 'halon2402|strat_o3', 'hcfc141b', 'hcfc141b|strat_o3', 'hcfc142b', 'hcfc142b|strat_o3', 'hcfc22', 'hcfc22|strat_o3', 'hfc125', 'hfc134a', 'hfc143a', 'hfc227ea', 'hfc23', 'hfc245fa', 'hfc32', 'hfc4310mee', 'methane', 'methane|strat_h2o', 'methane|trop_o3', 'methyl_bromide', 'methyl_bromide|strat_o3', 'methyl_chloride', 'methyl_chloride|strat_o3', 'nh3', 'nitrous_oxide', 'nmvoc', 'nmvoc|trop_o3', 'nox', 'nox|trop_o3', 'oc', 'oc|aci', 'sf6', 'so2', 'so2|aci'] 

100%|██████████| 535/535 [01:10<00:00,  7.56 timestep/s]


Integrating 7500 scenarios, 1 gas cycle parameter sets, 1 thermal response parameter sets, over ['bc', 'bc|aci', 'bc|bc_on_snow', 'c2f6', 'c6f14', 'carbon_dioxide', 'carbon_tetrachloride', 'carbon_tetrachloride|strat_o3', 'cf4', 'cfc11', 'cfc113', 'cfc113|strat_o3', 'cfc114', 'cfc114|strat_o3', 'cfc115', 'cfc115|strat_o3', 'cfc11|strat_o3', 'cfc12', 'cfc12|strat_o3', 'ch3ccl3', 'ch3ccl3|strat_o3', 'co', 'co|trop_o3', 'halon1202', 'halon1202|strat_o3', 'halon1211', 'halon1211|strat_o3', 'halon1301', 'halon1301|strat_o3', 'halon2402', 'halon2402|strat_o3', 'hcfc141b', 'hcfc141b|strat_o3', 'hcfc142b', 'hcfc142b|strat_o3', 'hcfc22', 'hcfc22|strat_o3', 'hfc125', 'hfc134a', 'hfc143a', 'hfc227ea', 'hfc23', 'hfc245fa', 'hfc32', 'hfc4310mee', 'methane', 'methane|strat_h2o', 'methane|trop_o3', 'methyl_bromide', 'methyl_bromide|strat_o3', 'methyl_chloride', 'methyl_chloride|strat_o3', 'nh3', 'nitrous_oxide', 'nmvoc', 'nmvoc|trop_o3', 'nox', 'nox|trop_o3', 'oc', 'oc|aci', 'sf6', 'so2', 'so2|aci'] 

100%|██████████| 535/535 [01:09<00:00,  7.66 timestep/s]


## Create FaIRv2.0.0 metadata

In [75]:
metadata = pd.DataFrame(columns = ['climate_model','climate_model_name','climate_model_version','climate_model_configuration_label','climate_model_configuration_description','project','name_of_person','literature_reference'],index=['entry'])
metadata.loc['entry','climate_model'] = 'FaIR'
metadata.loc['entry','climate_model_name'] = 'FaIRv2.0.0-alpha'
metadata.loc['entry','climate_model_version'] = '2.0.0'
metadata.loc['entry','climate_model_configuration_label'] = 'CONSTRAINED'
metadata.loc['entry','climate_model_configuration_description'] = 'FaIRv2.0.0 large parameter ensemble subset with observed level and rate constraint applied'
metadata.loc['entry','project'] = 'RCMIP'
metadata.loc['entry','name_of_person'] = 'Nicholas Leach <nicholas.leach@stx.ox.ac.uk>'
metadata.loc['entry','literature_reference'] = 'https://doi.org/10.5194/gmd-2019-379'

In [77]:
metadata.to_csv('/home/leachl/Documents/Simple_models/FaIR_v2-0/GIR/GIR_output/RCMIP/RCMIP_meta_FaIRv2.0.0-alpha.csv')

# Emission-driven experiments

## esm-hist

##### Tune r$_0$ to reproduce historical concs from the RCMIP emission series

In [155]:
## re-tune r0 to reproduce present-day concentrations:

## switch in GCP co2 emissions up to start of SSPs
GCP_historical_emissions=pd.read_csv('https://raw.githubusercontent.com/openclimatedata/global-carbon-budget/master/data/historical-budget.csv',index_col=0,usecols=[0,1,2])
## set 1750 value to 0
GCP_historical_emissions.loc[1750,'Fossil-Fuel-Industry'] = 0
## linearly interpolate between 0 and first land-use-change datapoint
GCP_historical_emissions.loc[1750:1849,'Land-Use-Change'] = np.linspace(0,0.99,100) * GCP_historical_emissions.loc[1850,'Land-Use-Change']

## import concentrations:
CMIP6_conc_ftp = ftplib.FTP('data.iac.ethz.ch','anonymous')
CMIP6_conc_ftp.cwd('CMIP6/input4MIPs/UoM/GHGConc/CMIP/yr/atmos/UoM-CMIP-1-1-0/GHGConc/gr3-GMNHSH/v20160701')
CMIP6_ftp_list = [x for x in CMIP6_conc_ftp.nlst() if x[-3:]=='csv']
WMGHG_concs = pd.DataFrame(dict(zip(['_'.join(x.split('_')[3:-8]) for x in CMIP6_ftp_list],[pd.read_csv('ftp://data.iac.ethz.ch/CMIP6/input4MIPs/UoM/GHGConc/CMIP/yr/atmos/UoM-CMIP-1-1-0/GHGConc/gr3-GMNHSH/v20160701/'+x,usecols=[0,1],index_col=0).iloc[:,0] for x in CMIP6_ftp_list])))
WMGHG_concs = WMGHG_concs[[x for x in WMGHG_concs.columns if x[-2:]!='eq']] # remove "equivalent" concentrations

WMGHG_concs['halon1202'] = 0
WMGHG_concs.loc[1765:2014,'halon1202'] = pd.read_csv('http://www.pik-potsdam.de/~mmalte/rcps/data/RCP45_MIDYEAR_CONCENTRATIONS.DAT',skiprows=38,delim_whitespace=True,index_col=0)['HALON1202'].loc[1765:2014].values

NOAA_mole_fractions_2020 = pd.read_csv('https://www.esrl.noaa.gov/gmd/aggi/NOAA_MoleFractions_2020.csv',skiprows=2,index_col=0,skipfooter=5).iloc[1:].replace('nd',np.nan).apply(pd.to_numeric)

scale_factor = (WMGHG_concs.loc[2014,'carbon_dioxide'] / NOAA_mole_fractions_2020.loc[2014:2015,'CO2']).values

CO2_concs = pd.Series(np.append(WMGHG_concs.loc[1750:,'carbon_dioxide'].values.flatten(),NOAA_mole_fractions_2020.loc[2015:,'CO2']*scale_factor),index=np.arange(1750,2020),name='CO2')

BE_GMST = pd.read_csv('http://berkeleyearth.lbl.gov/auto/Global/Land_and_Ocean_summary.txt',skiprows=48,index_col=0,usecols=[0,1],names=['Year','GMST'],delim_whitespace=True)
b, a = sp.signal.butter(2, 0.05)
BE_GMST['smooth'] = sp.signal.filtfilt(b, a, BE_GMST.values.flatten())

fit_concs = pd.concat([pd.DataFrame(CO2_concs)],keys=['observed'],axis=1)
fit_temp = BE_GMST['smooth']-BE_GMST.loc[1850:1900,'smooth'].mean()
## add on 100 years of data that interpolates the 1750-1850 temperature change with a linear fit
fit_temp = np.concatenate((np.zeros(100),fit_temp.values))

params = get_gas_parameter_defaults(['carbon_dioxide']).apply(pd.to_numeric)
a = params.loc['a1':'a4',('default','carbon_dioxide')].values
tau = params.loc['tau1':'tau4',('default','carbon_dioxide')].values
r = np.array([32.4,0.0177,2.64,0])
PI_conc = 278
emis2conc = params.loc['emis2conc',('default','carbon_dioxide')]

target = (((RCMIP_emms.loc[('World','historical','Emissions|CO2')].loc['1750':'2014'])/(1000*44.01/12.01)).sum())

def get_r0(x):
    r_1 = r.copy()
    r_1[0] = x
    emissions = invert_carbon_cycle_prescribed_T(fit_concs.values,fit_temp,a,tau,r_1,PI_conc,emis2conc)
    return (emissions.loc[:264].sum() - target)**2

print('r0:',sp.optimize.newton(get_r0,x0=30))

# print('r0:',36.02)

/home/leachl/miniconda3/lib/python3.7/site-packages/ipykernel_launcher.py:20: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support skipfooter; you can avoid this warning by specifying engine='python'.


r0: 33.81104011565178


In [20]:
ssp = 'historical'

SSP_emms = RCMIP_to_GIR_input_emms(ssp).interpolate(limit_area='inside').dropna()
SSP_emms -= SSP_emms.loc[1750]

SSP_LUC = pd.concat([get_RCMIP_forc([ssp],'Effective Radiative Forcing|Anthropogenic|Albedo Change')]*N,axis=1,keys=EXT_forc_sfs.loc['LUC'].index)*EXT_forc_sfs.loc['LUC'].values
SSP_other_forc = get_RCMIP_forc([ssp],['Effective Radiative Forcing|Natural'])
SSP_forc = (SSP_LUC + SSP_other_forc.values).loc[SSP_emms.index]

SSP_emms = pd.concat([SSP_emms]*N,axis=1,keys=EXT_forc_sfs.loc['LUC'].index)

emms_driven_params = gas_param_ensemble.copy()
# emms_driven_params.loc['r0',(slice(None),'carbon_dioxide')] *= 36.02/30.4

## run the model:
SSP_run = run_GIR(emissions_in=SSP_emms,forcing_in=SSP_forc,gas_parameters=emms_driven_params,thermal_parameters=thermal_param_ensemble)

Integrating 7500 scenarios, 1 gas cycle parameter sets, 1 thermal response parameter sets, over ['bc', 'bc|aci', 'bc|bc_on_snow', 'c2f6', 'c3f8', 'c4f10', 'c5f12', 'c6f14', 'c7f16', 'c8f18', 'c_c4f8', 'carbon_dioxide', 'carbon_tetrachloride', 'carbon_tetrachloride|strat_o3', 'cf4', 'cfc11', 'cfc113', 'cfc113|strat_o3', 'cfc114', 'cfc114|strat_o3', 'cfc115', 'cfc115|strat_o3', 'cfc11|strat_o3', 'cfc12', 'cfc12|strat_o3', 'ch2cl2', 'ch3ccl3', 'ch3ccl3|strat_o3', 'chcl3', 'co', 'co|trop_o3', 'halon1202', 'halon1202|strat_o3', 'halon1211', 'halon1211|strat_o3', 'halon1301', 'halon1301|strat_o3', 'halon2402', 'halon2402|strat_o3', 'hcfc141b', 'hcfc141b|strat_o3', 'hcfc142b', 'hcfc142b|strat_o3', 'hcfc22', 'hcfc22|strat_o3', 'hfc125', 'hfc134a', 'hfc143a', 'hfc152a', 'hfc227ea', 'hfc23', 'hfc236fa', 'hfc245fa', 'hfc32', 'hfc365mfc', 'hfc4310mee', 'methane', 'methane|strat_h2o', 'methane|trop_o3', 'methyl_bromide', 'methyl_bromide|strat_o3', 'methyl_chloride', 'methyl_chloride|strat_o3', 'nf3

100%|██████████| 264/264 [00:14<00:00, 18.30 timestep/s]


In [21]:
save_period=range(1750,2015)

## convert output to RCMIP formatting reqs:
T_out = pd.concat([SSP_run['T'].loc[:].copy()],axis=1,keys=[ssp],names=['Scenario','Gas cycle set'])
T_out = T_out.T.reset_index().rename({'Gas cycle set':'ensemble_member','Scenario':'scenario'},axis=1)
T_out.loc[:,'variable'] = 'Surface Air Temperature Change'
T_out.loc[:,'region'] = 'World'
T_out.loc[:,'climate_model'] = 'FaIRv2.0.0-alpha'
T_out.loc[:,'model'] = RCMIP_concs.loc[('World',ssp)].Model.unique()[0]
T_out.loc[:,'unit'] = 'K'
T_out = T_out.set_index(['climate_model','model','scenario','region','variable','unit','ensemble_member']).reset_index()

# get result category intersection with all possible categories:

RF_out = []
exp_forc_cats = [x for x,vals in param_categories.items() if bool(set(vals).intersection(SSP_run['RF'].columns.levels[-1]))]
for forc_cat in exp_forc_cats:
#     RF_out.loc[:,(slice(None),slice(None),forc_cat)] = 
    RF_out += [SSP_run['RF'].loc[:,(slice(None),param_categories[forc_cat])].groupby(level=0,axis=1).sum()]
RF_out += [SSP_run['RF'].loc[:].xs('Total',axis=1,level=-1)]
RF_out = pd.concat(RF_out,axis=1,keys=exp_forc_cats+['Effective Radiative Forcing'])
RF_out = pd.concat([RF_out],axis=1,keys=[ssp],names=['scenario','variable','ensemble_member'])
RF_out = RF_out.T.reset_index()
for forc_cat in exp_forc_cats:
    RF_out.replace(forc_cat,GIR_to_RCMIP_map_forc.loc[forc_cat,'RCMIP_forc_key'],inplace=True)
RF_out.loc[:,'region'] = 'World'
RF_out.loc[:,'climate_model'] = 'FaIRv2.0.0-alpha'
RF_out.loc[:,'model'] = RCMIP_concs.loc[('World',ssp)].Model.unique()[0]
RF_out.loc[:,'unit'] = 'W/m^2'
RF_out = RF_out.set_index(['climate_model','model','scenario','region','variable','unit','ensemble_member']).reset_index()

emms_out = SSP_run['Emissions'].loc[:,(slice(None),'carbon_dioxide')].apply(lambda x: x/GIR_to_RCMIP_map.loc[x.name[-1],'RCMIP_emms_scaling'],raw=False).copy()
emms_out = pd.concat([emms_out],axis=1,keys=[ssp],names=['scenario','ensemble_member','variable']).T.reset_index()
exp_emms_cats = [x for x in GIR_to_RCMIP_map.index if x in emms_out.variable.unique()]
for agent in exp_emms_cats:
    emms_out.replace(agent,GIR_to_RCMIP_map.loc[agent,'RCMIP_emms_key'],inplace=True)
emms_out.loc[:,'region'] = 'World'
emms_out.loc[:,'climate_model'] = 'FaIRv2.0.0-alpha'
emms_out.loc[:,'model'] = RCMIP_concs.loc[('World',ssp)].Model.unique()[0]
emms_out.loc[:,'unit'] = GIR_to_RCMIP_map.set_index('RCMIP_emms_key').loc[emms_out.variable,'RCMIP_emms_unit'].values
emms_out = emms_out.set_index(['climate_model','model','scenario','region','variable','unit','ensemble_member']).reset_index()

cum_emms_out = emms_out.loc[emms_out.variable=='Emissions|CO2'].copy()
atmos_pool_out = emms_out.loc[emms_out.variable=='Emissions|CO2'].copy()
af_out = emms_out.loc[emms_out.variable=='Emissions|CO2'].copy()
TCRE_out = emms_out.loc[emms_out.variable=='Emissions|CO2'].copy()

cum_emms_out = cum_emms_out.set_index(['climate_model','model','scenario','region','variable','unit','ensemble_member']).cumsum(axis=1).reset_index()
cum_emms_out.loc[:,'variable'] = 'Cumulative Emissions|CO2'

atmos_pool_out = atmos_pool_out.set_index(['climate_model','model','scenario','region','variable','unit','ensemble_member']).apply(lambda x: (SSP_run['C'].loc[:,(x.name[-1],'carbon_dioxide')]-278)/(4.688876e-01*GIR_to_RCMIP_map.loc['carbon_dioxide','RCMIP_emms_scaling']),axis=1).reset_index()
atmos_pool_out.loc[:,'variable'] = 'Carbon Pool|Atmosphere'
atmos_pool_out.loc[:,'unit'] = 'Mt CO2'

af_out = (atmos_pool_out.set_index(['climate_model','model','scenario','region','ensemble_member']).iloc[:,2:] / cum_emms_out.set_index(['climate_model','model','scenario','region','ensemble_member']).iloc[:,2:]).reset_index()
af_out.loc[:,'variable'] = 'Airborne Fraction|CO2'
af_out.loc[:,'unit'] = '-'
af_out = af_out.set_index(['climate_model','model','scenario','region','variable','unit','ensemble_member']).reset_index()

TCRE_out = (T_out.set_index(['climate_model','model','scenario','region','ensemble_member']).iloc[:,2:] / cum_emms_out.set_index(['climate_model','model','scenario','region','ensemble_member']).iloc[:,2:]).reset_index()
TCRE_out.loc[:,'variable'] = 'Instantaneous TCRE'
TCRE_out.loc[:,'unit'] = 'K / MtCO2'
TCRE_out = TCRE_out.set_index(['climate_model','model','scenario','region','variable','unit','ensemble_member']).reset_index()

CH4_tau = pd.concat([pd.concat([SSP_run['alpha'].xs('methane',axis=1,level=-1)*gas_param_ensemble.loc['tau1',(slice(None),'methane')].unique()[0]],axis=1,keys=['Atmospheric Lifetime|CH4'])],axis=1,keys=[ssp],names=['scenario','variable','ensemble_member'])
CH4_tau = CH4_tau.T.reset_index()
CH4_tau.loc[:,'region'] = 'World'
CH4_tau.loc[:,'climate_model'] = 'FaIRv2.0.0-alpha'
CH4_tau.loc[:,'model'] = RCMIP_concs.loc[('World',ssp)].Model.unique()[0]
CH4_tau.loc[:,'unit'] = 'yr'
CH4_tau = CH4_tau.set_index(['climate_model','model','scenario','region','variable','unit','ensemble_member']).reset_index()

output = pd.concat([T_out,RF_out,emms_out,cum_emms_out,atmos_pool_out,af_out,TCRE_out,CH4_tau],axis=0).loc[:,['climate_model','model','scenario','region','variable','unit','ensemble_member']+list(save_period)]

output.to_hdf(r'/home/leachl/Documents/Simple_models/FaIR_v2-0/GIR/GIR_output/RCMIP/RCMIP_esm-hist_FaIRv2.0.0-alpha.h5', key='stage', mode='w')

/home/leachl/miniconda3/lib/python3.7/site-packages/ipykernel_launcher.py:9: PerformanceWarning: indexing past lexsort depth may impact performance.
  if __name__ == '__main__':
/home/leachl/miniconda3/lib/python3.7/site-packages/ipykernel_launcher.py:28: PerformanceWarning: indexing past lexsort depth may impact performance.
/home/leachl/miniconda3/lib/python3.7/site-packages/ipykernel_launcher.py:39: PerformanceWarning: indexing past lexsort depth may impact performance.
/home/leachl/miniconda3/lib/python3.7/site-packages/ipykernel_launcher.py:69: PerformanceWarning: indexing past lexsort depth may impact performance.
/home/leachl/miniconda3/lib/python3.7/site-packages/pandas/io/pytables.py:282: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types that it cannot
map directly to c-types [inferred_type->integer,key->block1_items] [items->None]

  f(store)


## Saving memory

In [27]:
## reduce variables saved

choose_variable = ['Surface Air Temperature Change',\
                   'Effective Radiative Forcing|Anthropogenic|F-Gases',\
                   'Effective Radiative Forcing|Anthropogenic|Montreal Gases',\
                   'Effective Radiative Forcing|Anthropogenic|CO2',\
                   'Effective Radiative Forcing|Anthropogenic|CH4',\
                   'Effective Radiative Forcing|Anthropogenic|N2O',\
                   'Effective Radiative Forcing|Anthropogenic|Tropospheric Ozone',\
                   'Effective Radiative Forcing|Anthropogenic|Stratospheric Ozone',\
                   'Effective Radiative Forcing|Anthropogenic|Aerosols|Aerosols-radiation Interactions',\
                   'Effective Radiative Forcing|Anthropogenic|Aerosols|Aerosols-cloud Interactions',\
                   'Effective Radiative Forcing',\
                   'Emissions|CO2',\
                   'Carbon Pool|Atmosphere']

members = choose_mems[:5000]

def reduce_file_size(file,variables,ensemble_members,save_extension='_v1',single=True,end_date=2300):
    
    data_in = pd.read_hdf(file)
    
    if single:
        data_in.iloc[:,7:] = data_in.iloc[:,7:].astype(np.single)
        
    data_in = data_in.loc[(data_in.variable.isin(variables))&(data_in.ensemble_member.isin(ensemble_members))]
    
    data_in.replace('Surface Air Temperature Change','Surface Air Ocean Blended Temperature Change',inplace=True)
    
    data_in.loc[:,'ensemble_member'] = [x[3:] for x in data_in.loc[:,'ensemble_member']]
    
    final_data_date = data_in.columns[-1]
    
    if end_date<final_data_date:
        data_in = data_in.reindex(['climate_model','model','scenario','region','variable','unit','ensemble_member']+list(range(data_in.columns[7],end_date+1)),axis=1)
        
    data_in.to_csv('.'.join(file.split('.')[:-1])+save_extension+'.csv.gz', compression='gzip',index=False)

In [109]:
## also need to convert the surface air temp variable to blended surface air ocean temp

files = glob.glob('/home/leachl/Documents/Simple_models/FaIR_v2-0/GIR/GIR_output/RCMIP/*a.h5')

for file in files:
    if not ('idealised' in file or 'metrics' in file):
        reduce_file_size(file,choose_variable,members)
    
reduce_file_size('/home/leachl/Documents/Simple_models/FaIR_v2-0/GIR/GIR_output/RCMIP/RCMIP_idealised_FaIRv2.0.0-alpha.h5',choose_variable,members,end_date=150)

# Check Output

In [147]:
test = pd.read_csv('/home/leachl/Documents/Simple_models/FaIR_v2-0/GIR/GIR_output/RCMIP/upload/RCMIP_ssp245_FaIRv2.0.0-alpha_v1.csv.gz')

In [153]:
(test.loc[test.variable=='Surface Air Ocean Blended Temperature Change','2019']-test.loc[test.variable=='Surface Air Ocean Blended Temperature Change','2000']).describe(percentiles=[0.05,0.17,0.5,0.83,0.95])

count    5000.000000
mean        0.448323
std         0.101359
min         0.113393
5%          0.288371
17%         0.348234
50%         0.444614
83%         0.551179
95%         0.617148
max         0.850325
dtype: float64

In [134]:
# test.rename(dict(zip([str(x) for x in range(1,151)],[str(x) for x in range(1850,2000)])),axis=1).to_csv('/home/leachl/Documents/Simple_models/FaIR_v2-0/GIR/GIR_output/RCMIP/RCMIP_idealised_FaIRv2.0.0-alpha_v1.csv.gz', compression='gzip',index=False)